<a href="https://colab.research.google.com/github/carsonljc/APS360-Project/blob/master/Detector/Yolo_Image_Process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import os
import re
import glob
import os.path
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Unzip from drive
%%capture
!unzip /content/drive/'My Drive'/APS360/yolov3/train.zip -d /content/old_train/

In [ ]:
!pip install gdown

In [ ]:
#Unzip from link
!gdown --id 17vMkoPZSibult22BsN49Vm5osuROahaM --output sample.zip 
!unzip -q sample.zip -d /content/Mapillary

Downloading...
From: https://drive.google.com/uc?id=17vMkoPZSibult22BsN49Vm5osuROahaM
To: /content/sample.zip
15.9GB [04:08, 64.1MB/s]


In [ ]:
import torch
import torch.utils.data as data
from torch import nn
from torchvision import datasets, transforms

class AnnotationDataSet(data.Dataset):
    def __init__(self, path_to_imgs, path_to_json, transform = None):
        self.path_to_imgs = path_to_imgs
        self.path_to_json = path_to_json
        self.image_ids = os.listdir(path_to_imgs)
        self.transform = transform

    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_id = os.path.splitext(img_id)[0]
        img = Image.open(os.path.join(self.path_to_imgs, img_id + ".jpg"))
        json_file = json.load(open(os.path.join(self.path_to_json, img_id + ".json")))


        if self.transform:
            img = self.transform(img)

        return img, json_file, img_id

    def __len__(self):
        return len(self.image_ids)

In [ ]:
def construct_YOLO_labels (json_file, img_width, img_height):

  #Currently Assumes images are 3024 x 4032 (y, x)
  #TODO: Change hard code in this function!

  #Label in the format: [grid y coordinate][grid x coordinate][info]
  #info[0] - sign present, [1] - Bbox X, [2] - Bbox Y, [3] - Bbox W, [4] - Bbox H
  #all Bbox values are relative to size of a grid 
  #grid size currently 168x168 pixels 


  label = np.zeros([len(json_file["objects"]), 5])
  count = 0;

  for i in json_file["objects"]:

    x_center = (i["bbox"]['xmax'] - i["bbox"]['xmin'])/2 +  i["bbox"]['xmin']
    y_center = (i["bbox"]['ymax'] - i["bbox"]['ymin'])/2 + i["bbox"]['ymin']

    x_center = x_center / img_width
    y_center = y_center / img_height

    #x_index = int(x_center / grid_width)
    #y_index = int(y_center / grid_height)


    width = (i["bbox"]['xmax'] - i["bbox"]['xmin']) / img_width 
    height = (i["bbox"]['ymax'] - i["bbox"]['ymin']) / img_height

    #sign_type = -1
    #regulatory_check = "regulatory"
    #warning_check = "warning"
    #information_check = "information"
    #complementary_check = "complementary"
    #other_check = "other"
    #if (regulatory_check in i["label"]):
    #  sign_type = 0
    #elif (warning_check in  i["label"]):
    #  sign_type = 1
    #elif (information_check in i["label"]):
    #  sign_type = 2
    #elif (complementary_check in i["label"]):
    #  sign_type = 3
    #elif (other_check in i["label"]):
    #  sign_type = 4

    valid_signs = ['regulatory--no-parking--g2', 'regulatory--maximum-speed-limit-55--g2', 'warning--road-narrows-left--g2', 'regulatory--maximum-speed-limit-25--g2', 'warning--pedestrians-crossing--g4', 'warning--roundabout--g2', 'regulatory--keep-left--g2', 'regulatory--do-not-block-intersection--g1', 'regulatory--turn-left--g2', 'regulatory--stop--g1', 'warning--steep-descent--g2', 'complementary--obstacle-delineator--g1', 'regulatory--one-way-right--g3', 'warning--narrow-bridge--g1', 'warning--turn-right--g1', 'warning--t-roads--g2', 'regulatory--wrong-way--g1', 'warning--crossroads--g3', 'regulatory--maximum-speed-limit-30--g3', 'information--disabled-persons--g1', 'regulatory--go-straight--g3', 'regulatory--yield--g1', 'information--gas-station--g1', 'regulatory--no-heavy-goods-vehicles--g2', 'complementary--chevron-right--g1', 'regulatory--no-entry--g1', 'information--highway-exit--g1', 'complementary--tow-away-zone--g1', 'warning--traffic-merges-right--g1', 'regulatory--no-overtaking--g5', 'warning--junction-with-a-side-road-perpendicular-right--g3', 'warning--road-narrows--g2', 'regulatory--no-right-turn--g1', 'warning--turn-left--g1', 'warning--school-zone--g2', 'regulatory--no-straight-through--g1', 'information--hospital--g1', 'regulatory--no-u-turn--g1', 'regulatory--one-way-left--g3', 'warning--road-narrows-right--g2', 'regulatory--maximum-speed-limit-45--g3', 'complementary--both-directions--g1', 'regulatory--no-bicycles--g2', 'warning--winding-road-first-left--g1', 'regulatory--dual-lanes-go-straight-on-right--g1', 'regulatory--triple-lanes-turn-left-center-lane--g1', 'warning--traffic-merges-left--g1', 'warning--traffic-signals--g3', 'warning--y-roads--g1', 'information--telephone--g1', 'complementary--chevron-left--g1', 'regulatory--reversible-lanes--g2', 'warning--winding-road-first-right--g1', 'warning--double-curve-first-right--g2', 'regulatory--maximum-speed-limit-100--g3', 'regulatory--maximum-speed-limit-35--g2', 'regulatory--no-buses--g3', 'information--airport--g2', 'warning--curve-left--g2', 'warning--curve-right--g2', 'regulatory--turn-right--g3', 'warning--junction-with-a-side-road-perpendicular-left--g3', 'warning--double-curve-first-left--g2', 'regulatory--no-turn-on-red--g2', 'regulatory--dual-lanes-go-straight-on-left--g1', 'regulatory--no-left-turn--g2', 'regulatory--road-closed--g2', 'warning--divided-highway-ends--g2', 'warning--height-restriction--g2', 'complementary--obstacle-delineator--g2', 'regulatory--maximum-speed-limit-40--g3', 'information--bike-route--g1', 'regulatory--no-pedestrians--g2', 'regulatory--keep-right--g4', 'regulatory--go-straight-or-turn-left--g2']
    sign_type = 1

    for valid_sign in valid_signs:
      if (i["label"] in valid_sign):
        sign_type = 0
        break
    label[count] = np.array([sign_type, x_center, y_center, width, height])
    count += 1
  return label

In [ ]:
sample_set = AnnotationDataSet("/content/split_hand_selected/train/images", "/content/split_hand_selected/train/annotations", transform=transforms.ToTensor())

In [ ]:
%cd /content/split_hand_selected/train/images

/content/split_hand_selected/train/images


In [ ]:
import numpy as np
for img, json_file, img_id in iter(sample_set):
  #print (img.shape)
  label = construct_YOLO_labels(json_file, img.shape[2], img.shape[1])
  newline = 0
  file_name = img_id + ".txt"
  file = open(file_name, "w") 
  for i in label:
    #### 
    if (i[0] == 0): #Don't include "other-signs"
      if (newline):
        file.write("\n")
      #print (i)
      str_content = "{0:1d} {1:.6f} {2:.6f} {3:.6f} {4:.6f}".format(int(i[0]), i[1], i[2], i[3], i[4])
      file.write(str_content) 
      newline = 1
    ###
  file.close() 
  #print (label)

In [ ]:
class BoundingBox(object):
    def __init__(self, dim, image_size=None):
        # Assumes that dim are upper left corner and lower bottom corner
        if image_size is None:
            self.x1 = dim[0]
            self.y1 = dim[1]
            self.x2 = dim[2]
            self.y2 = dim[3]
        # Assumes we are using x_center, y_center, width, height
        else:
            x_center = dim[0] * image_size[0]
            y_center = dim[1] * image_size[1]
            width = dim[2] * image_size[0]
            height = dim[3] * image_size[1]

            self.x1 = x_center - width/2
            self.x2 = x_center + width/2
            self.y1 = y_center - height/2
            self.y2 = y_center + height/2
        
        assert self.x1 < self.x2
        assert self.y1 < self.y2

    def __str__(self):
        return "({},{},{},{})".format(self.x1, self.y1, self.x2, self.y2)
    
    def as_tuple(self):
        return (self.x1, self.y1, self.x2, self.y2)

    def to_yolo(self, bb2):
        x1 = max(self.x1, bb2.x1)
        x2 = min(self.x2, bb2.x2)
        y1 = max(self.y1, bb2.y1)
        y2 = min(self.y2, bb2.y2)
        bb1 = BoundingBox((x1, y1, x2, y2))

        width = bb2.x2 - bb2.x1
        height = bb2.y2 - bb2.y1
        
        x_center = ((bb1.x2 + bb1.x1)/2 - bb2.x1) / (bb2.x2 - bb2.x1)
        y_center = ((bb1.y2 + bb1.y1)/2 - bb2.y1) / (bb2.y2 - bb2.y1)
        width = (bb1.x2-bb1.x1) / (bb2.x2 - bb2.x1)
        height = (bb1.y2-bb1.y1) / (bb2.y2 - bb2.y1)
        assert x_center >= 0 and x_center <= 1
        assert y_center >= 0 and y_center <= 1
        assert width >= 0 and width <= 1
        assert height >= 0 and height <= 1
        return (x_center, y_center, width, height)

    # Thanks StackOverflow <3 : https://stackoverflow.com/a/42874377
    def get_iou(self, bb2):
        """
        Calculate the Intersection over Union (IoU) of two bounding boxes.

        Parameters
        ----------
        self : BoundingBox object
            Keys: {'x1', 'x2', 'y1', 'y2'}
            The (x1, y1) position is at the top left corner,
            the (x2, y2) position is at the bottom right corner
        bb2 : BoundingBox object
            Keys: {'x1', 'x2', 'y1', 'y2'}
            The (x, y) position is at the top left corner,
            the (x2, y2) position is at the bottom right corner

        Returns
        -------
        float
            in [0, 1]
        """
        assert self.x1 < self.x2
        assert self.y1 < self.y2
        assert bb2.x1 < bb2.x2
        assert bb2.y1 < bb2.y2

        # determine the coordinates of the intersection rectangle
        x_left = max(self.x1, bb2.x1)
        y_top = max(self.y1, bb2.y1)
        x_right = min(self.x2, bb2.x2)
        y_bottom = min(self.y2, bb2.y2)

        if x_right < x_left or y_bottom < y_top:
            return 0.0

        # The intersection of two axis-aligned bounding boxes is always an
        # axis-aligned bounding box
        intersection_area = (x_right - x_left) * (y_bottom - y_top)

        # compute the area of both AABBs
        self_area = (self.x2 - self.x1) * (self.y2 - self.y1)
        bb2_area = (bb2.x2 - bb2.x1) * (bb2.y2 - bb2.y1)

        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = intersection_area / self_area
        assert iou >= 0.0
        assert iou <= 1.0
        return iou



#Resize images to max_width, max_height while maintaining aspect ratio
#Discard small objects
def resize_image (input_files, max_width, max_height, output_dir, discard_dir, min_sign_pixel = 100):


    for image_file, bbox_file in input_files:
        obj_boxes = []
        # Calc Image size
        im = Image.open(image_file)
        imgwidth, imgheight = im.size

        #Ratios
        
        width_ratio = imgwidth / max_width
        height_ratio = imgheight / max_height

        #Check if we need to resize. If image is small, skip
        if (width_ratio < 1 and height_ratio < 1):
          continue

        #Resize by the larger ratio
        resize_ratio = width_ratio
        if (width_ratio < height_ratio):
          resize_ratio = height_ratio

        new_size = (int(imgwidth/resize_ratio), int(imgheight/resize_ratio))
        new_im = im.resize(new_size)

        # Get all the bounding boxes
        # Adjust bounding boxes if necessary
        with open(bbox_file, "r") as fp:
            name, _ = os.path.splitext(os.path.basename(bbox_file))
            #print ("Removed object in file %s:" % (name))
            for line in fp:
                # Look for image dimensions
                numbers = re.findall("[0-9.]+", line)
                numbers = [float(x) for x in numbers]
                if len(numbers):
                    bbox =  BoundingBox(numbers[1:], image_size=new_size)

                    #print (bbox.x2, bbox.x1, bbox.y2, bbox.y1, new_size)

                    #Check if the sign would still be visible after the resize
                    if ((bbox.x2 - bbox.x1) * (bbox.y2 - bbox.y1) < min_sign_pixel):
                      #print (numbers)
                      continue

                    obj_boxes.append(numbers[:])
                    #print (obj_boxes)


        
        if (len(obj_boxes) > 0):

          new_im.save(os.path.join(output_dir, "%s.jpg" % (name)))
          newline = 0  
          with open(os.path.join(output_dir, "%s.txt" % (name)), "w") as fp:
            for x in obj_boxes:
              if (newline):
                fp.write("\n")
              str_content = "{0:1d} {1:.6f} {2:.6f} {3:.6f} {4:.6f}".format(int(x[0]), x[1], x[2], x[3], x[4])
              fp.write(str_content)
              newline = 1
            fp.close()
        else:
          #print ("Image: ", name, " no longer has any detectable signs!")
          new_im.save(os.path.join(discard_dir, "%s.jpg" % (name)))



# Break up the images into multiple overlapping sections of lower resolution.
def break_up_image(input_files, size, stride, output_dir):
    for image_file, bbox_file in input_files:
        obj_boxes = []
        # Calc Image size
        im = Image.open(image_file)
        imgwidth, imgheight = im.size

        # Get all the bounding boxes
        with open(bbox_file, "r") as fp:
            name, _ = os.path.splitext(os.path.basename(bbox_file))
            for line in fp:
                # Look for image dimensions
                numbers = re.findall("[0-9.]+", line)
                numbers = [float(x) for x in numbers]
                if len(numbers):
                    obj_boxes.append(BoundingBox(numbers[1:], image_size=im.size))
        
        # Crop
        index = 0
        for i in range(0, imgheight, stride):
            for j in range(0, imgwidth, stride):
                # Out of bounds!
                if j+size >= imgwidth or i+size >= imgheight:
                    continue
                
                crop_box = BoundingBox((j, i, j+size, i+size))

                # Determine all valid object bounding boxes
              
                valid_obj_boxes = []
                for obj_box in obj_boxes:
                    if obj_box.get_iou(crop_box) >= 0.5:
                        valid_obj_boxes.append(obj_box)
                
                # if there is no valid box, then we skip
                if not valid_obj_boxes:
                    continue
                
                cropped = im.crop(crop_box.as_tuple())
                cropped.save(os.path.join(output_dir, "%s-%s.jpg" % (name, index)))

                newline = 0
                with open(os.path.join(output_dir, "%s-%s.txt" % (name, index)), "w") as fp:
                  #fp.writelines(["0 %s %s %s %s" % x.to_yolo(crop_box) for x in valid_obj_boxes])
                  #Might need to change the "0" to the actual number in the file in the future if we add more classes
                  for x in valid_obj_boxes:
                    if (newline):
                      fp.write("\n")   
                    yolo_bbox = x.to_yolo(crop_box)
                    str_content = "{0:1d} {1:.6f} {2:.6f} {3:.6f} {4:.6f}".format(0, yolo_bbox[0], yolo_bbox[1], yolo_bbox[2], yolo_bbox[3])
                    fp.write(str_content)
                    newline = 1
                  fp.close()
                    
                
                index +=1


In [ ]:
!rm -rf /content/resized/
!rm -rf /content/discard/

In [ ]:
os.makedirs("/content/resized", exist_ok=True)
os.makedirs("/content/discard", exist_ok=True)
#Sample_100/images/
print ("val")
bbox_list = sorted(glob.glob("/content/split_hand_selected/train/images/*.txt"))
images_list = sorted(glob.glob("/content/split_hand_selected/train/images/*.jpg"))
resize_image (zip(images_list, bbox_list), 1920, 1080, "/content/resized", "/content/discard", min_sign_pixel = 150)


val


In [ ]:
!rm -rf /content/val/

In [ ]:
os.makedirs("/content/train", exist_ok=True)


bbox_list = sorted(glob.glob("/content/resized/*.txt"))
images_list = sorted(glob.glob("/content/resized/*.jpg"))
break_up_image(zip(images_list, bbox_list), 416, 208, "/content/train")

In [ ]:
%cd /content

/content


In [ ]:
!ls

discard  drive	Hand_Selected  resized	sample_data  split_hand_selected  train


In [ ]:


!zip -q -r /content/drive/'My Drive'/APS360/yolov3/Datasets/hand_selected_416x416/train.zip train

!zip -q -r /content/drive/'My Drive'/APS360/yolov3/Datasets/hand_selected_416x416/train_discard.zip discard

Train, Val, and Test SPLITTING SECTION

In [ ]:
!gdown --id 1-uKo60uILmvaWTbSPVZGdP-hprLAbdd_ --output hand_selected.zip 


Downloading...
From: https://drive.google.com/uc?id=1-uKo60uILmvaWTbSPVZGdP-hprLAbdd_
To: /content/hand_selected.zip
11.8GB [02:58, 66.1MB/s]


In [ ]:
!unzip -q /content/hand_selected.zip -d /content/

In [ ]:
ls /content/Hand_Selected/images/ | wc -l

13641


In [ ]:
import os, os.path
base_dir = '/content/Hand_Selected/'
print("Annotation files: {}".format(len(os.listdir(base_dir+'annotations'))))
print("image files: {}".format(len(os.listdir(base_dir+'images'))))
print(os.listdir(base_dir+'annotations')[0])
print(os.listdir(base_dir+'images')[0])

Annotation files: 13641
image files: 13641
jobHCSXetCi0YHzCfO5ORg.json
S5u_ZPE9_LHSpEmM-qfq4A.jpg


In [ ]:
import os
from glob import glob
import shutil
from sklearn.model_selection import train_test_split

# find all image file names
image_files = glob(base_dir+'images/*.jpg')
image_names = [name.replace(".jpg","").split('/')[-1] for name in image_files]

# split names into 70% train, 15% val and 15% test
train_names, test_names = train_test_split(image_names, test_size=0.3,random_state=1)
val_names, test_names = train_test_split(test_names, test_size=0.5,random_state=1)
print(f'Train: {len(train_names)}, Val: {len(val_names)}, Test: {len(test_names)}')

Train: 9548, Val: 2046, Test: 2047


In [ ]:
print (train_names[0])

1UpXUk_d1pnnUpTH0u1bbA


In [ ]:
def move_files(file_names, source_path, destination_path):
    # create dir if not exist
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    if not os.path.exists(destination_path+'images'):
        os.makedirs(destination_path+'images')
    if not os.path.exists(destination_path+'annotations'):
        os.makedirs(destination_path+'annotations')
    
    # move images with given name
    for fi in file_names:
         image = fi+'.jpg'
         json = fi+'.json'
         
         shutil.move(os.path.join(source_path+'images', image), 
                     os.path.join(destination_path+'images', image))
         shutil.move(os.path.join(source_path+'annotations', json),
                     os.path.join(destination_path+'annotations', json))
    return

#CHANGE HERE
source_path = base_dir
destination_path = '/content/split_hand_selected/'
move_files(train_names,source_path,(destination_path+'train/'))
move_files(val_names,source_path,(destination_path+'val/'))
move_files(test_names,source_path,(destination_path+'test/'))

In [ ]:
%cd split_hand_selected/

/content/split_hand_selected


In [ ]:
!ls

test  train  val


In [ ]:
!zip -q -r /content/drive/'My Drive'/APS360/Project/Hand_Selected_Split/test.zip test 

In [ ]:
!zip -q -r /content/drive/'My Drive'/APS360/Project/Hand_Selected_Split/val.zip val 

In [ ]:
!zip -q -r /content/drive/'My Drive'/APS360/Project/Hand_Selected_Split/train.zip train 